In [1]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

In [2]:

#data_dir = '/home/juan/Desktop/juan/bio/mirna_mrcv/data/CT_SM1.2_BT1.2_V2.4/'
data_dir = '/home/juan/Desktop/juan/bio/mirna_mrcv/data/CT4_SM1.9_BT1.2_V2.4/'
#data_dir = '/home/juan/Desktop/juan/bio/mirna_mrcv/data/CT_old/'
res_dir = data_dir + 'res/'


path_data_dir = data_dir
path_res_dir = res_dir
path_fasta_res = res_dir + "mirna.deg.Y.fasta"
path_mirbase_res = res_dir + "mirna.y.mature.csv"
path_deg_file = path_res_dir + "mirna.deg.csv"
path_res_ann =  path_res_dir + "mirna.res.deg.ann.csv"

In [3]:
path_results = path_data_dir + "Results.txt"
df_res = pd.read_csv(path_results, sep="\t")
df_res.head(2)

,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,DicerCall,MIRNA,PhaseScore,Short,Long,20,21,22,23,24
0,1A:209-460,Cluster_1,252,127,0.997,64,1.0,+,CGCGGAUUGCUCGAGCUGCUCACGCGGCGAGAGCGGGUCG,25,...,N,N2,-1.0,30,80,3,1,5,4,4
1,1A:688-753,Cluster_2,66,86,0.675,0,1.0,+,CUCUAGACCGACUUUGUGAAAUGAC,5,...,N,N2,-1.0,7,61,5,1,4,4,4


In [4]:
df_deg = pd.read_csv(path_deg_file, sep=",")
print(len(df_deg.index))
df_deg.head(2)

13


,Name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,X21dpiR1C.trimmed,X21dpiR3C.trimmed,X21dpiR1T.trimmed,X21dpiR3T.trimmed
0,Cluster_5926,1555.831554,-0.932063,0.283616,-3.286356,0.001015,0.018472,2158.183752,1924.130447,861.401415,1279.610602
1,Cluster_6339,300.745016,-1.210371,0.460530,-2.628212,0.008583,0.071009,564.825185,275.042269,185.507795,177.604816


In [5]:
len(df_deg[df_deg.log2FoldChange > 0].index), len(df_deg[df_deg.log2FoldChange < 0].index)

(5, 8)

In [6]:
df_deg = df_deg[['Name','log2FoldChange']]

In [7]:
df_deg_y = pd.merge(df_res, df_deg, on='Name')
print(len(df_deg_y.index))
df_deg_y.head(2)

13


,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,MIRNA,PhaseScore,Short,Long,20,21,22,23,24,log2FoldChange
0,2A:485771022-485771245,Cluster_5926,224,6028,47.31,305,1.000,+,UAGGCGGUCACCUUGGCUAG,2878,...,Y,-1.0,1103,2,4052,813,54,1,3,-0.932063
1,2A:652328872-652329011,Cluster_6339,140,1185,9.30,1,0.001,-,UGCCAAAGGAGAGUUGCCCUG,1136,...,Y,-1.0,14,0,3,1155,11,2,0,-1.210371


In [8]:
buffer_seqs = []
for k,v in df_deg_y.iterrows():
    description = '%s l2fc:%f' % (v['#Locus'], v['log2FoldChange'])
    description = ''
    seq = SeqRecord(Seq(v.MajorRNA), id=v.Name, description=description)
    buffer_seqs.append(seq)
SeqIO.write(buffer_seqs, path_fasta_res, "fasta")
print(len(buffer_seqs))
print(path_fasta_res)

13
/home/juan/Desktop/juan/bio/mirna_mrcv/data/CT4_SM1.9_BT1.2_V2.4/res/mirna.deg.Y.fasta


In [9]:
cmd = "blastn -task blastn -evalue 10 -query %s -subject data/mature.fa -outfmt '6 qseqid sseqid qstart qend sstart send mismatch gaps pident evalue length qlen slen qcovs score' > %s"
cmd = cmd % (path_fasta_res, path_mirbase_res)
cmd

"blastn -task blastn -evalue 10 -query /home/juan/Desktop/juan/bio/mirna_mrcv/data/CT4_SM1.9_BT1.2_V2.4/res/mirna.deg.Y.fasta -subject data/mature.fa -outfmt '6 qseqid sseqid qstart qend sstart send mismatch gaps pident evalue length qlen slen qcovs score' > /home/juan/Desktop/juan/bio/mirna_mrcv/data/CT4_SM1.9_BT1.2_V2.4/res/mirna.y.mature.csv"

In [57]:
df_mirbase = pd.read_csv(path_mirbase_res, sep="\t")
cols = ['qseqid','sseqid','qstart','qend','sstart','send','mismatch','gaps','pident','evalue','length','qlen','slen','qcovs','score']
df_mirbase.columns = cols
print(len(df_mirbase.index))
df_mirbase.head(2)


497


,qseqid,sseqid,qstart,qend,sstart,send,mismatch,gaps,pident,evalue,length,qlen,slen,qcovs,score
0,Cluster_5926,sbi-miR169d-3p,3,20,2,19,0,0,100.000,0.00021,18,20,20,90,36
1,Cluster_5926,bdi-miR169h-3p,3,20,1,18,1,0,94.444,0.00900,18,20,19,90,30


In [58]:
df_mirbase[df_mirbase.qseqid=='Cluster_30794'].head(4)

,qseqid,sseqid,qstart,qend,sstart,send,mismatch,gaps,pident,evalue,length,qlen,slen,qcovs,score


In [59]:
allowed_mirbase_mismatches = 4
mirbase_qlen = 4

In [60]:
df_mirbase = df_mirbase[df_mirbase.length >= df_mirbase.qlen - mirbase_qlen]
print(len(df_mirbase.index))


447


In [61]:
df_mirbase = df_mirbase[df_mirbase.mismatch + df_mirbase.gaps <= allowed_mirbase_mismatches]
print(len(df_mirbase.index))


447


In [62]:
len(df_mirbase.qseqid.unique().tolist())


12

In [63]:
df_mirbase_tae = df_mirbase[df_mirbase.sseqid.str.startswith('tae')]
df_mirbase_tae.head(20)

,qseqid,sseqid,qstart,qend,sstart,send,mismatch,gaps,pident,evalue,length,qlen,slen,qcovs,score
176,Cluster_6339,tae-miR399,1,19,1,19,1,0,94.737,0.002000,19,21,19,90,32
217,Cluster_6540,tae-miR9776,1,20,2,21,1,0,95.000,0.000650,20,21,21,95,34
265,Cluster_10580,tae-miR9776,1,20,2,21,1,0,95.000,0.000650,20,21,21,95,34
458,Cluster_17722,tae-miR169,2,22,1,22,0,1,95.455,0.000743,22,22,22,95,34
466,Cluster_19633,tae-miR9674b-5p,1,20,1,20,1,0,95.000,0.000650,20,21,21,95,34
467,Cluster_19633,tae-miR9674a-5p,4,21,1,18,1,0,94.444,0.008000,18,21,21,86,30
476,Cluster_27288,tae-miR169,2,22,1,22,0,1,95.455,0.000743,22,22,22,95,34
484,Cluster_33786,tae-miR5200,1,21,1,21,0,0,100.000,0.000004,21,21,21,100,42
491,Cluster_35773,tae-miR5200,1,21,1,21,0,0,100.000,0.000004,21,21,21,100,42


In [64]:
df_mirbase_tae = df_mirbase_tae.loc[df_mirbase_tae.groupby("qseqid")["score"].idxmax()]
df_mirbase_tae.head(20)

,qseqid,sseqid,qstart,qend,sstart,send,mismatch,gaps,pident,evalue,length,qlen,slen,qcovs,score
265,Cluster_10580,tae-miR9776,1,20,2,21,1,0,95.000,0.000650,20,21,21,95,34
458,Cluster_17722,tae-miR169,2,22,1,22,0,1,95.455,0.000743,22,22,22,95,34
466,Cluster_19633,tae-miR9674b-5p,1,20,1,20,1,0,95.000,0.000650,20,21,21,95,34
476,Cluster_27288,tae-miR169,2,22,1,22,0,1,95.455,0.000743,22,22,22,95,34
484,Cluster_33786,tae-miR5200,1,21,1,21,0,0,100.000,0.000004,21,21,21,100,42
491,Cluster_35773,tae-miR5200,1,21,1,21,0,0,100.000,0.000004,21,21,21,100,42
176,Cluster_6339,tae-miR399,1,19,1,19,1,0,94.737,0.002000,19,21,19,90,32
217,Cluster_6540,tae-miR9776,1,20,2,21,1,0,95.000,0.000650,20,21,21,95,34


In [65]:
df_mirbase_rest = df_mirbase[~df_mirbase.index.isin(df_mirbase_tae.index)]
df_mirbase_rest = df_mirbase.loc[df_mirbase.groupby("qseqid")["score"].idxmax()]


In [66]:
print(len(df_mirbase_tae.index))
print(len(df_mirbase_rest.index))
df_mirbase_tae.head(2)


8
12


,qseqid,sseqid,qstart,qend,sstart,send,mismatch,gaps,pident,evalue,length,qlen,slen,qcovs,score
265,Cluster_10580,tae-miR9776,1,20,2,21,1,0,95.000,0.000650,20,21,21,95,34
458,Cluster_17722,tae-miR169,2,22,1,22,0,1,95.455,0.000743,22,22,22,95,34


In [67]:
df_mirbase_tae = df_mirbase_tae[['qseqid','sseqid']]
df_mirbase_tae.rename(columns={'sseqid':'mirbase_tae'}, inplace=True)
df_mirbase_rest = df_mirbase_rest[['qseqid','sseqid']]
df_mirbase_rest.rename(columns={'sseqid':'mirbase_rest'}, inplace=True)
df_mirbase_tae.head(2)


,qseqid,mirbase_tae
265,Cluster_10580,tae-miR9776
458,Cluster_17722,tae-miR169


In [68]:
df_deg_y.head(2)

,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,MIRNA,PhaseScore,Short,Long,20,21,22,23,24,log2FoldChange
0,2A:485771022-485771245,Cluster_5926,224,6028,47.31,305,1.000,+,UAGGCGGUCACCUUGGCUAG,2878,...,Y,-1.0,1103,2,4052,813,54,1,3,-0.932063
1,2A:652328872-652329011,Cluster_6339,140,1185,9.30,1,0.001,-,UGCCAAAGGAGAGUUGCCCUG,1136,...,Y,-1.0,14,0,3,1155,11,2,0,-1.210371


In [69]:
df_1 = pd.merge(df_deg_y, df_mirbase_tae, how='left', left_on='Name', right_on='qseqid')
df_1.drop(['qseqid'], axis=1, inplace=True)
print(len(df_1.index))
df_1.head(2)


13


,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,PhaseScore,Short,Long,20,21,22,23,24,log2FoldChange,mirbase_tae
0,2A:485771022-485771245,Cluster_5926,224,6028,47.31,305,1.000,+,UAGGCGGUCACCUUGGCUAG,2878,...,-1.0,1103,2,4052,813,54,1,3,-0.932063,NaN
1,2A:652328872-652329011,Cluster_6339,140,1185,9.30,1,0.001,-,UGCCAAAGGAGAGUUGCCCUG,1136,...,-1.0,14,0,3,1155,11,2,0,-1.210371,tae-miR399


In [70]:
df_2 = pd.merge(df_1, df_mirbase_rest, how='left', left_on='Name', right_on='qseqid')
df_2.drop(['qseqid'], axis=1, inplace=True)
print(len(df_2.index))
df_2.head(2)

13


,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,Short,Long,20,21,22,23,24,log2FoldChange,mirbase_tae,mirbase_rest
0,2A:485771022-485771245,Cluster_5926,224,6028,47.31,305,1.000,+,UAGGCGGUCACCUUGGCUAG,2878,...,1103,2,4052,813,54,1,3,-0.932063,NaN,sbi-miR169d-3p
1,2A:652328872-652329011,Cluster_6339,140,1185,9.30,1,0.001,-,UGCCAAAGGAGAGUUGCCCUG,1136,...,14,0,3,1155,11,2,0,-1.210371,tae-miR399,aof-miR399a


In [71]:
df_2['mirbase'] = df_2['mirbase_tae'].fillna(df_2['mirbase_rest'])
df_2.drop(['mirbase_tae','mirbase_rest'], axis=1, inplace=True)
df_2.head(3)

,#Locus,Name,Length,Reads,RPM,UniqueReads,FracTop,Strand,MajorRNA,MajorRNAReads,...,PhaseScore,Short,Long,20,21,22,23,24,log2FoldChange,mirbase
0,2A:485771022-485771245,Cluster_5926,224,6028,47.310,305,1.000,+,UAGGCGGUCACCUUGGCUAG,2878,...,-1.0,1103,2,4052,813,54,1,3,-0.932063,sbi-miR169d-3p
1,2A:652328872-652329011,Cluster_6339,140,1185,9.300,1,0.001,-,UGCCAAAGGAGAGUUGCCCUG,1136,...,-1.0,14,0,3,1155,11,2,0,-1.210371,tae-miR399
2,2A:707747476-707747579,Cluster_6540,104,1330,10.438,1,0.999,+,UGGACGAGGAUGUGCAGCUGC,1050,...,-1.0,8,2,0,1316,4,0,0,0.975894,tae-miR9776


In [72]:
df_2.to_csv(path_res_ann, sep="\t", index=None)
path_res_ann

'/home/juan/Desktop/juan/bio/mirna_mrcv/data/CT4_SM1.9_BT1.2_V2.4/res/mirna.res.deg.ann.csv'